In [ ]:
import os
import time
import win32com.client
import PySimpleGUI as sg
from pathlib import Path

#functions to open and close workbooks (deletes excel.exe in task manager on close)
def open_workbook(fileLocation):
    if os.path.exists(fileLocation):
        global xl, wb
        xl=win32com.client.Dispatch('Excel.Application')
        wb = xl.Workbooks.Open(os.path.abspath(fileLocation))
        xl.Application.Visible = True   
def close_and_save_workbook():
    for wb in xl.workbooks:
        wb.Close(SaveChanges=1)
    del  globals()['xl']
    time.sleep(1)

# Window #1 (Share_Reports folder path)
layout = [[sg.Text('Path to Scenario_Chooser folder:')],
          [sg.Listbox(values=('C:/Users/jimmy.gov/Desktop/Share_Reports', 'C:/Users/bryant.vu/Documents/Share_Reports'), size=(80, 5))],
          [sg.OK(), sg.Cancel()]]
window = sg.Window('Share Reports', layout)

while True:
    event, values = window.Read()
    if event is None or event == 'Exit' or event == 'Cancel':
        exit = True
        break
    else:
        excel_files_path = values[0][0]
        gif_file_path = values[0][0] + '/.gif Files/monkey.gif'
        excel_files_list = os.listdir(excel_files_path)
        excel_files_list = [x for x in excel_files_list if '.xlsm' in x]
        excel_files_list.insert(0, 'Run All')
        break
window.Close()

# Window #2 (User options)
layout = [[sg.Text('Excel file:')],
          [sg.Listbox(values=excel_files_list, size=(80, 10))],
          [sg.Text('[Required] Enter year_month date (YYYYMM):'), sg.InputText()],
          [sg.Text('[Required] Enter report date (YYYY.MM.DD):'), sg.InputText()],
          [sg.Text('MTD?'), sg.Combo(['TRUE', 'FALSE'], default_value = 'TRUE')],
          [sg.Text('Refresh Cube?'), sg.Combo(['Yes', 'No'], default_value = 'No')],
          [sg.OK(), sg.Cancel()]]
window = sg.Window('Share Reports', layout)

while True:
    event, values = window.Read()
    if event is None or event == 'Exit' or event == 'Cancel':
        break
    else:
        # Create inputs dictionary 
        inputs_list = ['file_name', 'year_month', 'report_date', 'MTD','refresh_cube']
        inputs_dict = {}
        for i, input in enumerate(inputs_list):
            if i == 0:
                inputs_dict[input] = values[i][0]
            else:
                inputs_dict[input] = values[i]
        if inputs_dict['file_name'] == 'Run All':
            for file_name in excel_files_list[1:]:
                open_workbook(excel_files_path + '/' + file_name)
                if inputs_dict['refresh_cube'] == 'Yes':
                    xl.Run('update_slicer', inputs_dict['year_month'], inputs_dict['report_date'], inputs_dict['MTD'])
                    xl.Run('runAll')
                    close_and_save_workbook()
                else:
                    xl.Run('runAll')
                    close_and_save_workbook()
                
        else:
            open_workbook(excel_files_path + '/' + inputs_dict['file_name'])
            if inputs_dict['refresh_cube'] == 'Yes':
                xl.Run('update_slicer', inputs_dict['year_month'], inputs_dict['report_date'], inputs_dict['MTD'])
                xl.Run('runAll')
                close_and_save_workbook()
            else:
                xl.Run('runAll')
                close_and_save_workbook()
        break
